In [8]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
import sys
sys.path.append('../../GlobalRepo/')
from global_repo import *

client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [9]:
program_id = LOUNGE_PROGRAM_ID

sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)


In [10]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')
        

payer_keypair = payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
payer_keypair.public_key

2ZHc9QxDDaJwqNEFzpAGUrvxCWZSNnXSffHxV9hG2axp

In [64]:
table_mint_id = PublicKey("FXyQAAPKHqxGcKtF9jPnchV1aL2iUdEfBEkkKpJhr6i1")
payer_table_associated_id = instructions.get_associated_token_address(payer_keypair.public_key, table_mint_id)
(freeze_authority, nonce) = PublicKey.find_program_address([b"table_mint_authority_pda"], TABLES_PROGRAM_ID)
table_data_pda, _ = PublicKey.find_program_address([b"table_data_pda", bytes(table_mint_id)], TABLES_PROGRAM_ID)
lounge_pda, _ = PublicKey.find_program_address([b"Tables_Lounge"], program_id)
authorizer_pda, _ = PublicKey.find_program_address([b"authorizer_pda"], program_id)


payer_account_meta = AccountMeta(payer_keypair.public_key, True, False)
table_mint_meta = AccountMeta(table_mint_id, False, True)
payer_table_associated_meta = AccountMeta(payer_table_associated_id, False, True)
freeze_authority_meta = AccountMeta(freeze_authority, False, False)
table_data_pda_meta = AccountMeta(table_data_pda, False, True)
lounge_pda_meta = AccountMeta(lounge_pda, False, True)
authorizer_pda_meta = AccountMeta(authorizer_pda, False, False)
table_program_meta = AccountMeta(TABLES_PROGRAM_ID, False, False)




In [65]:
accounts = [
    payer_account_meta, 
    table_mint_meta,
    payer_table_associated_meta,
    freeze_authority_meta,
    table_data_pda_meta,
    lounge_pda_meta,
    authorizer_pda_meta,
    spl_program_meta,

    
    table_program_meta,
]
accounts

[AccountMeta(pubkey=2ZHc9QxDDaJwqNEFzpAGUrvxCWZSNnXSffHxV9hG2axp, is_signer=True, is_writable=False),
 AccountMeta(pubkey=FXyQAAPKHqxGcKtF9jPnchV1aL2iUdEfBEkkKpJhr6i1, is_signer=False, is_writable=True),
 AccountMeta(pubkey=vxkJFw2VgsU3Xz5JemxXjPMmkJSEHEs2fYGN35dPnHk, is_signer=False, is_writable=True),
 AccountMeta(pubkey=DdCFLwf3fc6XexU7KPDYqKzum11yXKtqB2pFGEvuMTyG, is_signer=False, is_writable=False),
 AccountMeta(pubkey=HDTWHBVcPsnauREX1wGoRjpXWvigbL4pRoTm8pf4x16h, is_signer=False, is_writable=True),
 AccountMeta(pubkey=5A9kxy6RK2xTGtZMboWvQhLqifWqfzPdxV9tYZ6mSosJ, is_signer=False, is_writable=True),
 AccountMeta(pubkey=FLmj5tyHqLGnJ64SomBCpuPWhGnD5qpP1v8TrTHFRPU3, is_signer=False, is_writable=False),
 AccountMeta(pubkey=TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA, is_signer=False, is_writable=False),
 AccountMeta(pubkey=7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz, is_signer=False, is_writable=False)]

In [69]:
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([0])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: invalid account data for instruction', 'data': {'accounts': None, 'err': {'InstructionError': [0, 'InvalidAccountData']}, 'logs': ['Program 53toELc2ZRKUQKcbCcpq3JfUxu6CamAfKWThxbHuiwCP invoke [1]', 'Program log: right before table data deserialization', 'Program log: table data indicates table is already on the lobby, an issue stems there', 'Program 53toELc2ZRKUQKcbCcpq3JfUxu6CamAfKWThxbHuiwCP consumed 13742 of 200000 compute units', 'Program 53toELc2ZRKUQKcbCcpq3JfUxu6CamAfKWThxbHuiwCP failed: invalid account data for instruction'], 'unitsConsumed': 0}}

In [68]:
accountSchema = CStruct(
    "all_tables" / Vec(U8[32])
)

getAccountInfo(client, lounge_pda) 

Container(all_tables=ListContainer([ListContainer([215, 242, 207, 230, 28, 123, 83, 227, 149, 242, 223, 49, 192, 71, 132, 223, 72, 128, 119, 24, 29, 10, 156, 64, 192, 195, 93, 191, 128, 70, 54, 134])]))